In [134]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
import statsmodels.api as sm
from sklearn.metrics import confusion_matrix, classification_report,f1_score,accuracy_score,precision_score,recall_score
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from time import time
import timeit #imports timeit module

In [51]:
df = pd.read_csv('Data/Cataract Data 1.3.csv', index_col=0, parse_dates=True)
df.head()

,ogAA_letter,positionAA_num,mutAA_letter,iso_point,ogAA_char,mutAA_char,deldel_G1,solv_area,deldel_G2,deldel_G3,evo_age,pdel,positive_negative
index_name,,,,,,,,,,,,,
A171T,A,171,T,6.76,n,l,-0.47,85.3,-0.550085,-0.005249,324,0.50,positive
D140N,D,140,N,7.15,-,l,-0.96,59.0,-1.864259,-0.180476,750,0.74,positive
D109H,D,109,H,7.18,-,"H,+",-0.42,65.2,-0.661944,-0.509662,750,0.74,positive
D109A,D,109,A,7.15,-,n,-0.11,65.2,-0.550471,-0.670734,750,0.74,positive
P20S,P,20,S,6.76,P,l,-0.78,35.9,-0.703610,-0.456211,750,0.74,positive


In [153]:
def ML_logistic_regression_summaries(*argv):
    
    Vars = []
    
    for arg in argv:
        Vars = Vars + [arg]
        
    X = df.loc[:, Vars] #student; balance; income
    y = df.loc[:, "positive_negative"] #default
    
    
    model = LogisticRegression(solver= 'liblinear')

    from time import time
    import timeit #imports timeit module
    start_time = timeit.default_timer() #defines start time so computational time can be calculated

    acc_score = [];
    Truth = [];
    Output = [];

    for train_index , test_index in kf.split(df):
        #print(train_index); ## to check the training index
        #print(test_index); ## to check the testing index
        #print(); print();

        X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = .1)

        #print([X_train.shape, y_train.shape]);
        #print([X_test.shape, y_test.shape]);

        model.fit(X_train,y_train)
        pred_values = model.predict(X_test)

        acc = accuracy_score(pred_values, y_test)
        acc_score.append(acc)

        Truth.extend(y_test.values.reshape(y_test.shape[0])); ## it is a list
        Output.extend(pred_values); ## it is a list  


    dfOutput = pd.DataFrame(data={"X Variable(s)": [str(Vars),],"Avg Accuracy": [scores.mean(),],"SD of Accuracy": [np.std(scores),],"Sensitivity": [recall_score(Truth,Output,pos_label="negative"),],"Precision": [precision_score(Truth,Output,pos_label="negative"),],"F1-Score": [f1_score(Truth,Output,pos_label="negative"),], "Runtime": [elapsed,]}) #,"Real Specificity": [1,]
    
    return dfOutput

In [152]:
ML_logistic_regression_summaries("pdel","deldel_G3")

,X Variable(s),Avg Accuracy,SD of Accuracy,Sensitivity,Precision,F1-Score,Runtime
0,"['pdel', 'deldel_G3']",0.625,0.484123,0.969388,0.605096,0.745098,0.127439
